# tif to png converting

In [ ]:
# from glob import glob
# import pandas as pd
# import cv2
# from tqdm import tqdm

# load_images = glob('./raw_data/*/images/*.*')
# load_csv = glob('./raw_data/*/*.*')

# save_path = './raw_data_png/'

# for img_path in tqdm(load_images):
#     img = cv2.imread(img_path)
    
#     img_name = '/'.join(img_path.split('/')[-3:])[:-4]+'.png'
#     cv2.imwrite(save_path+img_name, img)
    

# def _tif_to_csv(path): return path[:-4]+'.png' 
# for csv_path in load_csv :
#     df =pd.read_csv(csv_path, header=None)
#     df[0] = df[0].apply(_tif_to_csv)
    
#     df.to_csv(save_path + '/'.join(csv_path.split('/')[2:]),index = None)

In [ ]:
os.path.join('/'.join(csv_path.split('/')[:-1]), 'images')

# CSV 데이터 유효성 확인

In [ ]:
import pandas as pd
from glob import glob
import os

# csv validation 확인
# annotation과 이미지 정보가 일치하는지 확인

def csv_valid_check(csv_path):
    print("label\t","n_csv\t","n_images")
    def _extract_img_name(path): return path.split('/')[-1]

    label = csv_path.split('/')[-2]

    df = pd.read_csv(csv_path)

    images = list(map(_extract_img_name,glob(os.path.join('/'.join(csv_path.split('/')[:-1]))+'/*.jpg')))
    csv_images = df['img_path'].unique()

    print("{}\t{}\t{}".format(label, len(csv_images), len(images)))
    print(list(set(csv_images)^set(images)))
    print("\n\n")
    
csv_path_list = glob('/home/jovyan/DATA/googlemap/google_data/*/*.csv')
for csv_path in csv_path_list:
    csv_valid_check(csv_path)

In [ ]:
# 시각화
import random
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os

def display_anno_samples(df_path, n_samples=5):
    data_df = pd.read_csv(df_path,names=['img_path','x1','y1','x2','y2','label'])

    images = list(data_df['img_path'].unique())
    images = random.sample(images,n_samples)
    
    for img_path in images:    
        img = cv2.imread(os.path.join('/'.join(df_path.split('/')[:-1]),img_path))
        draw_img = img.copy()

        anno_info = data_df[data_df['img_path']==img_path]
        n_obj = len(anno_info.values)

        for x1, y1, x2, y2,label in anno_info[['x1','y1','x2','y2','label']].values:
            draw_img = cv2.rectangle(draw_img, (int(x1),int(y1)),(int(x2),int(y2)),(255,0,0),1)

        print ("number of objects : {}".format(n_obj))
        plt.figure(figsize=(10,10))
        plt.title(label)
        plt.imshow(draw_img)
        plt.show()
        

df_path = '/home/jovyan/DATA/googlemap/google_data/13/13.csv'
display_anno_samples(df_path, n_samples=3)

# 

# CSV Annotation Merge & 절대경로 추가

In [ ]:
import pandas as pd

def merge_csv_to_df(csv_path_list, save_path):
    
    column_names=['img_path','x1','y1','x2','y2','label']
    merged_df = pd.DataFrame(columns=column_names)
    
    for csv_path in csv_path_list:
        df = pd.read_csv(csv_path)
        merged_df = merged_df.append(df)
        
    merged_df.to_csv(save_path, index=None)
    print("{} annotations from {} images are stored".format(len(merged_df), len(merged_df['img_path'].unique())))

    
csv_path_list = glob('/home/jovyan/DATA/googlemap/google_data/*/*.csv')
merge_csv_to_df(csv_path_list, '/home/jovyan/DATA/googlemap/google_data/label.csv')

In [ ]:
# img_path 절대경로 변경
import pandas as pd
import os

def set_abs_path(df,img_root_path, save_path):
    anno_df = df.copy()
    for idx, (img_path, label) in enumerate(anno_df[['img_path','label']].values):
         anno_df['img_path'].iloc[idx] = os.path.abspath(os.path.join(img_root_path,str(label),img_path))
    anno_df.to_csv(save_path,index=None)
    
data_df = pd.read_csv('/home/jovyan/DATA/googlemap/google_data/label.csv')
set_abs_path(data_df,'/home/jovyan/DATA/googlemap/google_data/','/home/jovyan/DATA/googlemap/google_data/label_abspath.csv')

## 

# 데이터셋 Train / Test Split

In [ ]:
import pandas as pd
import random
import time

def split_dataset(df_path, test_ratio=0.2):
    data_df = pd.read_csv(df_path)
    split_df = data_df.copy()

    labels_category = {}
    for idx, label in enumerate(sorted(split_df['label'].unique())): labels_category[label] = idx

    def set_values(row, value): return value[row]
    split_df['category_id'] = split_df['label'].apply(set_values, args =(labels_category, ))
    split_df['is_train'] = True

    for label in sorted(data_df['label'].unique()):
        img_path_list = list(data_df[data_df['label'] == label]['img_path'].unique())
        test_img_path = random.sample(img_path_list,int(len(img_path_list)*test_ratio))
        
        for test_img in test_img_path:
            indexes = list(data_df[data_df['img_path']==test_img].index)
            split_df.loc[indexes,['is_train']] = False
        
    # check numbers
    print("label\t total\t train\t test")
    for label in sorted(data_df['label'].unique()):
        n_total = len(data_df[data_df['label']==label])
        n_train = len(split_df[(split_df['label']==label) & (split_df['is_train']==True)])
        n_test = len(split_df[(split_df['label']==label) & (split_df['is_train']==False)])
        print ("{}\t {}\t {}\t {}".format(label, n_total, n_train, n_test))
    
    return split_df

df_path = '/home/jovyan/DATA/googlemap/google_data/label_abspath.csv'
split_df = split_dataset(df_path, 0.2)

split_df.to_csv('/home/jovyan/DATA/googlemap/csv/label.csv',index=None)

# 

# CSV annotation를 모델 학습용 데이터로 변경
- torch_yolov3 학습 포멧으로 변경
- tf_yolov3 학습 포멧으로 변경

#### 

### torch_yolov3 포멧
- 이미지 당 .txt 파일을 pair로 annotation 생성

In [ ]:
import pandas as pd
import shutil
import cv2
from tqdm import tqdm
import os
from glob import glob

def convert_anno_to_torch_yolov3(df_path, text_save_path):
    data_df = pd.read_csv(df_path)
    
    labels = data_df['label'].unique()
    mapping = {}
    for idx, label in enumerate(sorted(labels)): mapping[label] = idx
    
    # directory setup
    try : shutil.rmtree(text_save_path)
    except: pass
    os.mkdir(text_save_path)
    for label in labels: os.mkdir(os.path.join(text_save_path,str(label)))
    
    for img_path, x1, y1, x2, y2, label,_,_ in tqdm(data_df.values):
        img_name = img_path.split('/')[-1][:-4]
        
        img = cv2.imread(img_path)
        h,w,_ = img.shape
            
        with open(os.path.join(text_save_path, str(label),img_name+'.txt'),'a') as f:
            cx = ((int(x1) + int(x2)) // 2) / w
            cy = ((int(y1) + int(y2)) // 2) / h
            cw = (int(x2) - int(x1)) / w
            ch = (int(y2) - int(y1)) / h

            category_id = mapping[int(label)]

            f.write(f"{category_id} {cx} {cy} {cw} {ch}\n")

            
# 저장 폴더 디렉토리 생성
df_path = '/home/jovyan/DATA/googlemap/csv/label.csv'
text_save_path = '/home/jovyan/DATA/googlemap/torch/labels' # './format_data/torch_yolov3/labels'
convert_anno_to_torch_yolov3(df_path, text_save_path)

In [ ]:
def valid_txt_annotation(df_path, text_save_path):
    # csv와 txt의 annotation 갯수 확인
    data_df = pd.read_csv(df_path)
    labels = sorted(data_df['label'].unique())

    print("label\t n_csv\t n_txt")
    for label in labels:
        txt_files = glob(os.path.join(text_save_path, str(label))+'/*.*')
        n_txt_anno = 0    

        for txt_path in txt_files: 
            f_name = txt_path.split('/')[-1][:-4]

            with open(txt_path,'r') as f :
                lines = f.readlines()
                n_txt_anno +=len(lines)

        print("{}\t {}\t {}\t".format(label,len(data_df[data_df['label']==label]), n_txt_anno))
    
valid_txt_annotation(df_path, text_save_path)

In [ ]:
import cv2
import pandas as pd
import os
import shutil

def save_train_test_set(df_path, save_root_path, anno_txt_path):
    data_df = pd.read_csv(df_path)

    # directory setup
    try : shutil.rmtree(save_root_path)
    except: pass
    os.mkdir(save_root_path)
    os.mkdir(os.path.join(save_root_path,'train'))
    os.mkdir(os.path.join(save_root_path,'train','images'))
    os.mkdir(os.path.join(save_root_path,'train','labels'))
    os.mkdir(os.path.join(save_root_path,'test'))
    os.mkdir(os.path.join(save_root_path,'test','images'))
    os.mkdir(os.path.join(save_root_path,'test','labels'))

    for img_path in data_df['img_path'].unique():
        f_name = img_path.split('/')[-1]

        is_train = data_df[data_df['img_path']==img_path]['is_train'].unique()[0]
        label = data_df[data_df['img_path']==img_path]['label'].unique()[0]

        txt_path = os.path.join(anno_txt_path,str(label),f_name[:-4]+'.txt')

        if is_train : save_path = os.path.join(save_root_path,'train')
        else: save_path = os.path.join(save_root_path,'test')

        shutil.copy(txt_path, os.path.join(save_path,'labels',f_name[:-4]+'.txt'))
        shutil.copy(img_path, os.path.join(save_path,'images',f_name))    
        
df_path = '/home/jovyan/DATA/googlemap/csv/label.csv'
save_root_path = '/home/jovyan/DATA/googlemap/torch/data/'
anno_txt_path = '/home/jovyan/DATA/googlemap/torch/labels/'
save_train_test_set(df_path, save_root_path, anno_txt_path)

print("train images : ",len(glob('/home/jovyan/DATA/googlemap/torch/data/train/images/*.*')))
print("test images : ",len(glob('/home/jovyan/DATA/googlemap/torch/data/test/images/*.*')))

print("train annotations : ",len(glob('/home/jovyan/DATA/googlemap/torch/data/train/labels/*.*')))
print("test annotations : ",len(glob('/home/jovyan/DATA/googlemap/torch/data/test/labels/*.*')))

#  

### tf_yolov3 포멧
- tfrecord로 변환

In [1]:
import cv2
import pandas as pd
import tensorflow as tf

In [ ]:
def _set_tf_features(anno_df,img_shape):
    img_path = anno_df['img_path'].unique()
    if len(img_path) > 1: return warnings.warn("Image annotation is wrong")
    
    img_raw = open(img_path[0], 'rb').read()
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    classes_text = []
    height,width = img_shape

    for x1,y1,x2,y2,label in anno_df[['x1','y1','x2','y2','label']].values:
        xmin.append(float(x1) / width)
        ymin.append(float(y1) / height)
        xmax.append(float(x2) / width)
        ymax.append(float(y2) / height)
        
        classes_text.append(str(label).encode('utf8'))
        

    features={
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_path[0].encode('utf8')])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmin)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmax)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymin)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymax)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        }
    
    example = tf.train.Example(features=tf.train.Features(feature=features))
    
    return example

In [ ]:
import cv2
import pandas as pd
from tqdm import tqdm

def create_tfrecord_from_csv(data_df, output_path):
    writer = tf.io.TFRecordWriter(output_path)
    img_list = data_df['img_path'].unique()
    
    for img_path in tqdm(img_list):
        img = cv2.imread(img_path)
        df = data_df[data_df['img_path']==img_path]
        
        tfrecord_example = _set_tf_features(df,img.shape[:2])
        writer.write(tfrecord_example.SerializeToString())
    writer.close()

In [2]:
df_path = '/home/jovyan/DATA/googlemap/csv/label.csv'

train_df = df[df['is_train']==True].copy()
test_df = df[df['is_train']==False].copy()

create_tfrecord_from_csv(train_df, '/home/jovyan/DATA/googlemap/tfrecord/train2.tfrecord')
create_tfrecord_from_csv(test_df, '/home/jovyan/DATA/googlemap/tfrecord/test2.tfrecord')


# 

## COCO dataset

In [ ]:
import pandas as pd
import json
import cv2
from tqdm import tqdm

In [ ]:
def data_to_coco(anno_df, destfile):

    anno_df["label"] = anno_df["label"].astype(str)
    
    
    label_names = list(anno_df['label'].unique())
    label_names = [str(x) for x in label_names]
    
    label_categories = {}
    
    # label 별 ID 부여
    for label in label_names:
        label_categories[label] = int(anno_df[anno_df["label"]==label]['category_id'].unique()[0])
    
    data_dict = {}
    data_dict['images'] = []
    data_dict['categories'] = []
    data_dict['annotations'] = []
    
    for idx, name in enumerate(label_names):
        single_category = {'id': label_categories[name], 'name': name, 'supercategory': name}
        data_dict['categories'].append(single_category)

    inst_count = 1
    image_id = 1
    
    with open(destfile, 'w') as f_out:
        images = list(anno_df['img_path'].unique())
        
        for image_path in tqdm(images): 
            img_name = image_path.split('/')[-1]
            
            img = cv2.imread(image_path)
            height, width, _  = img.shape

            single_image = {}
            single_image['file_name'] = img_name
            single_image['id'] = image_id
            single_image['width'] = width
            single_image['height'] = height
            data_dict['images'].append(single_image)

            # annotations
            objects = anno_df[anno_df['img_path']==image_path]
        
            for x1, y1, x2, y2,label in objects[["x1", "y1", "x2", "y2","label"]].values:
                single_obj = {}

                single_obj['category_id'] = label_categories[label]

                width, height = x2-x1, y2-y1
                
                single_obj['bbox'] = x1, y1, width, height
                single_obj['area'] = width*height
                
                single_obj['image_id'] = image_id
                
                single_obj['iscrowd'] = 0
                single_obj['ignore'] = 0
                
                data_dict['annotations'].append(single_obj)
                single_obj['id'] = inst_count
                inst_count = inst_count + 1
            image_id = image_id + 1
            
        json.dump(data_dict, f_out)

In [ ]:
anno_df = pd.read_csv('/home/jovyan/DATA/googlemap/csv/label.csv')

data_to_coco(anno_df[anno_df['is_train']==True], '/home/jovyan/DATA/googlemap/coco/train.json')
data_to_coco(anno_df[anno_df['is_train']==False], '/home/jovyan/DATA/googlemap/coco/test.json')